In [ ]:
# --- Cell 1: Imports & Setup ---
import os
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import rasterio
from rasterio.mask import mask
from shapely.geometry import box, mapping
import shapely.ops
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.stats import mode
import warnings
warnings.filterwarnings("ignore")

# 📁 Folder paths
input_dir = "/home/jovyan/mystorage/NPP_inputs/"
output_dir = "/home/jovyan/mystorage/NPP_outputs/"
os.makedirs(output_dir, exist_ok=True)

print("✅ Environment ready.")


In [ ]:
# --- Cell 2: Load the ESTK LUT from Excel ---
lut_path = "/home/jovyan/mystorage/25-04-15_conversietabel_MODIS_ESTK_LUTable_voorlopig.xlsx"

# 📥 Load, strip whitespace, and set index to 'Pixelwaarde'
lut_df = pd.read_excel(lut_path)
lut_df.columns = lut_df.columns.str.strip()  # clean column names
lut_df = lut_df.set_index("Pixelwaarde")
lut_df.index = lut_df.index.astype(int)  # ensure proper matching

# ✅ Preview
print("📋 LUT loaded with classes:", lut_df.index.tolist())
lut_df.head()


In [ ]:
# --- Cell 3: Load and align input rasters ---

# 🌱 LAI & FAPAR
lai = rxr.open_rasterio(f"{input_dir}LAI_Gelderland.tif", masked=True).squeeze() / 10000
fapar = rxr.open_rasterio(f"{input_dir}FAPAR_Gelderland.tif", masked=True).squeeze() / 10000

# 🌡 ERA5 Temperature (°C)
temp = rxr.open_rasterio(f"{input_dir}temp_Gelderland_May2022.tif", masked=True).squeeze()

# ☀️ ERA5 DSSF (Daily Surface Solar Radiation)
dssf = rxr.open_rasterio(f"{input_dir}dssf_Gelderland_May2022.tif", masked=True).squeeze()

# ✅ Align all rasters to LAI grid
temp = temp.rio.reproject_match(lai)
dssf = dssf.rio.reproject_match(lai)
fapar = fapar.rio.reproject_match(lai)

# 🧪 Confirm alignment
print("📐 Shapes (y, x):")
print("LAI:", lai.shape)
print("FAPAR:", fapar.shape)
print("TEMP:", temp.shape)
print("DSSF:", dssf.shape)


In [ ]:
# --- Cell 4: Load and align ESTK raster to LAI grid ---

# Load the clipped ESTK map (already subset to Gelderland)
estk_path = f"{input_dir}ESTK_Gelderland.tif"
estk_clip = rxr.open_rasterio(estk_path, masked=True).squeeze()

# Reproject to match LAI (and thus all other rasters)
estk_clip_aligned = estk_clip.rio.reproject_match(lai)

# Check unique pixel classes present
unique_vals = np.unique(estk_clip_aligned.values[~np.isnan(estk_clip_aligned.values)])
print("🧩 ESTK classes found in raster:", unique_vals)


In [ ]:
# --- Cell 5: NPP model loop: compute NPP per ecosystem class ---

# 🎯 Filter only the classes present in both ESTK raster and LUT
valid_classes = [int(v) for v in unique_vals if int(v) in lut_df.index]
print(f"✅ Will compute NPP for {len(valid_classes)} ecosystem classes.")

# 📦 Initialize empty NPP array
npp_array = xr.full_like(lai, np.nan)

# 💡 Loop over each valid ecosystem type
for pixelwaarde in valid_classes:
    try:
        # Mask for the current ESTK class
        mask = estk_clip_aligned == pixelwaarde
        if mask.sum() == 0:
            print(f"⚠️ Pixelwaarde {pixelwaarde} not found in aligned raster — skipping.")
            continue

        # Retrieve parameter values from LUT
        row = lut_df.loc[pixelwaarde]
        eps_max = row["eps_max"]
        t_min = row["T_min"]
        t_max = row["T_max"]

        # Apply mask to input variables
        temp_masked = temp.where(mask)
        fapar_masked = fapar.where(mask)
        dssf_masked = dssf.where(mask)

        # Final sanity check (skip if all NaNs)
        if np.isnan(temp_masked).all():
            print(f"⚠️ All values masked out for class {pixelwaarde}, skipping.")
            continue

        # 🌿 NPP Calculation
        temp_factor = ((temp_masked - t_min) * (t_max - temp_masked)) / ((t_max - t_min) ** 2)
        temp_factor = temp_factor.clip(min=0)

        gpp = eps_max * fapar_masked * dssf_masked
        npp = gpp * temp_factor

        # Merge into output
        npp_array = npp_array.where(~mask, npp)

        print(f"✅ Processed Pixelwaarde {pixelwaarde}")

    except Exception as e:
        print(f"❌ Error processing {pixelwaarde}: {e}")


In [ ]:
# --- Cell 6: Save final NPP result to GeoTIFF ---

# Define output path
npp_out_path = f"{output_dir}NPP_Gelderland_May2022.tif"

# Ensure CRS is attached before export
npp_array.rio.write_crs(lai.rio.crs, inplace=True)

# Write to GeoTIFF
npp_array.rio.to_raster(npp_out_path)

print(f"✅ NPP raster saved to: {npp_out_path}")


In [ ]:
# --- Cell 7: Side-by-side NPP + ESTK plot with LUT-based legend ---
from matplotlib import colors as mcolors

# ⬇️ Reduce resolution for visualization (to avoid overload)
npp_plot = npp_array.coarsen(x=10, y=10, boundary="trim").mean()

# 🧠 Use mode for categorical ESTK downsampling
def xr_mode_2d(arr, axis):
    # Ensures mode across both (y, x)
    return mode(arr, axis=axis, nan_policy='omit').mode

estk_plot = estk_clip_aligned.coarsen(x=10, y=10, boundary="trim").reduce(xr_mode_2d)

# 🎨 Colormap setup for ESTK
unique_vals = np.unique(estk_plot.values[~np.isnan(estk_plot.values)])
unique_vals = [int(v) for v in unique_vals]

cmap = plt.cm.tab20
norm = mcolors.BoundaryNorm(boundaries=np.arange(min(unique_vals)-0.5, max(unique_vals)+1.5), ncolors=len(unique_vals))

# 🔖 Build legend
estk_labels = lut_df["ESTK omschrijving"].to_dict()
legend_labels = [estk_labels.get(v, f"Class {v}") for v in unique_vals]
legend_colors = [cmap(norm(v)) for v in unique_vals]
handles = [
    plt.Line2D([0], [0], marker='s', color='none', markerfacecolor=c, label=l, markersize=8)
    for c, l in zip(legend_colors, legend_labels)
]

# 🖼️ Plot NPP and ESTK side by side
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# NPP
npp_plot.plot(ax=axes[0], cmap="YlGn", robust=True)
axes[0].set_title("🌱 NPP (May 2022)")

# ESTK
im = axes[1].imshow(estk_plot, cmap=cmap, norm=norm)
axes[1].set_title("🧩 Ecosysteemtype (ESTK)")
axes[1].legend(handles=handles, loc='lower left', bbox_to_anchor=(1.05, 0), title="Legend", fontsize='small')

plt.tight_layout()
plt.show()


In [ ]:
# --- Cell 8: Notes and Guidance for Scaling the Workflow ---

print("🧭 Notes for Scaling the NPP Pipeline:\n")

print("1️⃣ Change Region (Province or AOI):")
print("- Update the ESTK raster clip (e.g., Utrecht, Overijssel, etc.)")
print("- Update LAI and FAPAR tiles for the matching Sentinel-2 grid")
print("- Clip temperature & DSSF rasters accordingly")

print("\n2️⃣ Change Time Period:")
print("- Update LAI/FAPAR to a different month (e.g., June)")
print("- Extract new ERA5 layers for TEMP and DSSF (monthly mean or daily sum)")
print("- Update output filenames accordingly")

print("\n3️⃣ Extend to Full Netherlands:")
print("- Consider using a tile-based loop (multiple Sentinel-2 UTM zones)")
print("- May need to mosaic LAI/FAPAR first and work with chunked rasters")
print("- Use cloud computing (e.g. CDSE, Terrascope, WEkEO) for scalability")

print("\n4️⃣ Export Options:")
print("- Use `.rio.to_raster()` to save intermediate layers (e.g. LAI_combined)")
print("- Load into QGIS or use `geopandas` to summarize by admin units")
print("- Add PNG or PDF export of plots with `plt.savefig()`")

print("\n✅ All code cells are modular — can be adapted per region/month.")
print("🏁 Done! 🚀")
